In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
lst = []

for i in range(10000):
    lst.append([str(i),"name"+str(i)])

In [4]:
lst_rdd = spark.sparkContext.parallelize(lst)

In [7]:
lst_df = spark.createDataFrame(lst_rdd,["id","name"])

In [8]:
lst_df.show(5)

+---+-----+
| id| name|
+---+-----+
|  0|name0|
|  1|name1|
|  2|name2|
|  3|name3|
|  4|name4|
+---+-----+
only showing top 5 rows



In [9]:
lst_df.rdd.getNumPartitions()

4

In [10]:
lst_df.coalesce(1).write.mode("overwrite").saveAsTable("unbuket_demo")

In [11]:
lst_df.coalesce(1).write.bucketBy(4,"id").sortBy("id").mode("overwrite").saveAsTable("buket_demo")

In [12]:
df_read1 = spark.table("unbuket_demo")
df_read1.where("id='1000'").show()

+----+--------+
|  id|    name|
+----+--------+
|1000|name1000|
+----+--------+



In [13]:
df_read = spark.table("buket_demo")
df_read.where("id='1000'").show()

+----+--------+
|  id|    name|
+----+--------+
|1000|name1000|
+----+--------+



In [15]:
df_read1.where("id='1000'").explain()

== Physical Plan ==
*(1) Project [id#26, name#27]
+- *(1) Filter (isnotnull(id#26) && (id#26 = 1000))
   +- *(1) FileScan parquet default.unbuket_demo[id#26,name#27] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/C:/Users/admin/Documents/spark_dataAnalytics_machine_learning/spark-wareh..., PartitionFilters: [], PushedFilters: [IsNotNull(id), EqualTo(id,1000)], ReadSchema: struct<id:string,name:string>


In [16]:
df_read.where("id='1000'").explain()

== Physical Plan ==
*(1) Project [id#37, name#38]
+- *(1) Filter (isnotnull(id#37) && (id#37 = 1000))
   +- *(1) FileScan parquet default.buket_demo[id#37,name#38] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/C:/Users/admin/Documents/spark_dataAnalytics_machine_learning/spark-wareh..., PartitionFilters: [], PushedFilters: [IsNotNull(id), EqualTo(id,1000)], ReadSchema: struct<id:string,name:string>, SelectedBucketsCount: 1 out of 4


In [17]:
spark.sql("describe extended unbuket_demo").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|                  id|              string|   null|
|                name|              string|   null|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|            Database|             default|       |
|               Table|        unbuket_demo|       |
|        Created Time|Mon Dec 07 12:29:...|       |
|         Last Access|Thu Jan 01 05:29:...|       |
|          Created By|         Spark 2.4.4|       |
|                Type|             MANAGED|       |
|            Provider|             parquet|       |
|            Location|file:/C:/Users/ad...|       |
+--------------------+--------------------+-------+



In [18]:
spark.sql("describe extended buket_demo").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|                  id|              string|   null|
|                name|              string|   null|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|            Database|             default|       |
|               Table|          buket_demo|       |
|        Created Time|Mon Dec 07 12:33:...|       |
|         Last Access|Thu Jan 01 05:29:...|       |
|          Created By|         Spark 2.4.4|       |
|                Type|             MANAGED|       |
|            Provider|             parquet|       |
|         Num Buckets|                   4|       |
|      Bucket Columns|              [`id`]|       |
|        Sort Columns|              [`id`]|       |
|            Location|file:/C:/Users/ad...|       |
+--------------------+--------------------+-------+

